## Loading Data

In [2]:
using DelimitedFiles

In [52]:
data = readdlm("caveMap.txt",String);

In [54]:
caveMap = split.(data,"")
caveMap = transpose(parse.(Int,reduce(hcat,caveMap)));

In [76]:
caveMapTest = [2 1 9 9 9 4 3 2 1 0; 3 9 8 7 8 9 4 9 2 1; 9 8 5 6 7 8 9 8 9 2; 8 7 6 7 8 9 6 7 8 9; 9 8 9 9 9 6 5 6 7 8]

5×10 Array{Int64,2}:
 2  1  9  9  9  4  3  2  1  0
 3  9  8  7  8  9  4  9  2  1
 9  8  5  6  7  8  9  8  9  2
 8  7  6  7  8  9  6  7  8  9
 9  8  9  9  9  6  5  6  7  8

## Part 1

In [55]:
map = caveMap
largeMap= fill(10,(size(map).+2))
largeMap[2:end-1,2:end-1] = map
largeMap;

In [56]:
riskLevels = 0
nx,ny = size(largeMap)
mask = [0 1 0; 1 0 1; 0 1 0]
for i in 2:nx-1, j in 2:ny-1
    maskSum=sum((largeMap[i-1:i+1,j-1:j+1].*mask).>largeMap[i,j])
    if maskSum == 4
        #println("i $(i) j $(j) value $(largeMap[i,j])")
        riskLevels += (largeMap[i,j]+1)
    end
end
riskLevels

480

## Part 2

In [96]:
function recursiveBasinMapping(map,numBasin,i,j)
    #paint this cell as belonging to basin numBasin
    map[i,j] = numBasin
    ##check if neighboring cell is 0 (so has not been processes and is not boundary)
    if map[i-1,j] == 0     ##up
        recursiveBasinMapping(map,numBasin,i-1,j)
    end
    if map[i,j-1] == 0     ##left
        recursiveBasinMapping(map,numBasin,i,j-1)
    end    
    if map[i,j+1] == 0     ##right 
        recursiveBasinMapping(map,numBasin,i,j+1)
    end
    if map[i+1,j] == 0     ##down
        recursiveBasinMapping(map,numBasin,i+1,j)
    end  
end

function findBasins(caveMap)
    
    #bulid larger map with boundary values at edges
    largeMap= fill(9,(size(caveMap).+2))
    largeMap[2:end-1,2:end-1] = caveMap
    # basin boundary map
    map = (largeMap.==9)* -1  
    
    mapProcessed = false
    numBasins = 0
    while !mapProcessed
        indx = findfirst(x->x==0,map)
        if isnothing(indx)
            mapProcessed = true # map has been processed, exit loop
        else
            numBasins += 1
            recursiveBasinMapping(map,numBasins,indx[1],indx[2])
        end
    end
    
    ## find largest basins
    basinSizes = Array{Int}(undef,numBasins)
    for i in 1:numBasins
        basinSizes[i] = count(x->x==i,map)
    end
    
    sort!(basinSizes)
    
    return (numBasins,map,basinSizes)
end

findBasins (generic function with 1 method)

### Input

In [103]:
nbasins,basinMap,basinSizes = findBasins(caveMap);

In [104]:
println(basinSizes)
prod(basinSizes[end-2:end])

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 8, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 11, 11, 11, 12, 12, 12, 13, 13, 13, 14, 14, 14, 15, 15, 15, 15, 15, 15, 16, 16, 17, 17, 17, 17, 18, 18, 18, 19, 19, 19, 19, 20, 20, 21, 21, 22, 22, 22, 23, 23, 23, 24, 24, 24, 25, 25, 25, 25, 25, 26, 26, 26, 28, 28, 29, 29, 30, 30, 30, 30, 31, 31, 32, 32, 32, 33, 33, 33, 34, 34, 34, 35, 35, 35, 36, 36, 36, 36, 37, 38, 38, 38, 38, 38, 38, 39, 39, 40, 40, 40, 40, 40, 41, 41, 41, 42, 42, 42, 43, 44, 44, 44, 44, 44, 45, 45, 45, 45, 45, 46, 47, 47, 48, 48, 50, 50, 50, 50, 51, 52, 53, 53, 54, 54, 55, 57, 58, 58, 58, 62, 62, 62, 62, 63, 63, 63, 66, 67, 68, 68, 69, 71, 71, 72, 73, 74, 75, 75, 76, 76, 77, 79, 79, 81, 82, 84, 87, 88, 88, 88, 89, 89, 92, 95, 97, 98, 110]


1045660

In [106]:
nbasins

226

### Test input

In [97]:
nbasinsTest,basinMapTest,basinSizesTest = findBasins(caveMapTest);

In [94]:
basinMapTest

7×12 Array{Int64,2}:
 -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1
 -1   1   1  -1  -1  -1   3   3   3   3   3  -1
 -1   1  -1   2   2   2  -1   3  -1   3   3  -1
 -1  -1   2   2   2   2   2  -1   4  -1   3  -1
 -1   2   2   2   2   2  -1   4   4   4  -1  -1
 -1  -1   2  -1  -1  -1   4   4   4   4   4  -1
 -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1

In [98]:
basinSizesTest

4-element Array{Int64,1}:
  3
  9
  9
 14

In [100]:
prod(basinSizesTest[end-2:end])

1134